In [2]:
# General python libraires 
import os

# General DS libraries
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 

# Parallelized libraries 
import dask.dataframe as dd 

In [3]:
# Getting files 
path = 'C:/Users/redha.cherif_artefac/GitHub_perso/American_express_comp/input/'
train = dd.read_csv(os.path.join(path, 'train_data.csv'))
train_labels = dd.read_csv(os.path.join(path, 'train_labels.csv'))
test = dd.read_csv(os.path.join(path, 'test_data.csv'))

In [18]:
print(len(train))
print(len(test))

5531451
11363762


In [45]:
print(dd.compute(train.S_2.min(),train.S_2.max()))
print(dd.compute(test.S_2.min(), test.S_2.max()))
print('train: 12 months')
print('test: 18 months')

('2017-03-01', '2018-03-31')
('2018-04-01', '2019-10-31')
train: 12 months
test: 18 months


In [15]:
train_cust_ID = train.customer_ID.unique().compute()
train_labels_custID = train_labels.customer_ID.unique().compute()
test_cust_ID = test.customer_ID.unique().compute()

In [21]:
print(f'Number of customers in train : {len(train_cust_ID)}')
print(f'Number of customers in test : {len(test_cust_ID)}')

Number of customers in train : 458913
Number of customers in test : 924621


In [53]:
train_test_join = pd.merge(train_cust_ID,test_cust_ID,how='inner')
train_trainLabel_join = pd.merge(train_cust_ID,train_labels_custID,how='inner')
print(f'Number of common customers in the train and the test : {len(train_test_join)}')
print(f'Number of common customers in the train and the train_labels : {len(train_trainLabel_join)}')
print('')
print('Therefore, all customers in the test are different from the ones in the train ! No leakage possible at first glance.')
print('The customers in the train_labels are the same than in the train')

Number of common customers in the train and the test : 0
Number of common customers in the train and the train_labels : 458913

Therefore, all customers in the test are different from the ones in the train ! No leakage possible at first glance.
The customers in the train_labels are the same than in the train


In [4]:
cat_feats = train[['customer_ID','B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']]
cat_feats.head()

,customer_ID,B_30,B_38,D_114,D_116,D_117,D_120,D_126,D_63,D_64,D_66,D_68
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.0,2.0,1.0,0.0,4.0,0.0,1.0,CR,O,NaN,6.0
1,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.0,2.0,1.0,0.0,4.0,0.0,1.0,CR,O,NaN,6.0
2,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.0,2.0,1.0,0.0,4.0,0.0,1.0,CR,O,NaN,6.0
3,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.0,2.0,1.0,0.0,4.0,0.0,1.0,CR,O,NaN,6.0
4,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.0,2.0,1.0,0.0,4.0,0.0,1.0,CR,O,NaN,6.0


In [83]:
print(f'Unique values for B_30: {cat_feats.B_30.unique().compute()}')
print(f'Unique values for B_38: {cat_feats.B_38.unique().compute()}')
print(f'Unique values for D_114: {cat_feats.D_114.unique().compute()}')
print(f'Unique values for D_116: {cat_feats.D_116.unique().compute()}')
print(f'Unique values for D_117: {cat_feats.D_117.unique().compute()}')
print(f'Unique values for D_120: {cat_feats.D_120.unique().compute()}')
print(f'Unique values for D_126: {cat_feats.D_126.unique().compute()}')
print(f'Unique values for D_63: {cat_feats.D_63.unique().compute()}')
print(f'Unique values for D_64: {cat_feats.D_64.unique().compute()}')
print(f'Unique values for D_66: {cat_feats.D_66.unique().compute()}')
print(f'Unique values for D_68: {cat_feats.D_68.unique().compute()}')

Unique values for B_30: 0    0.0
1    2.0
2    1.0
3    NaN
Name: B_30, dtype: float64
Unique values for B_38: 0    2.0
1    1.0
2    3.0
3    5.0
4    6.0
5    7.0
6    4.0
7    NaN
Name: B_38, dtype: float64
Unique values for D_114: 0    1.0
1    0.0
2    NaN
Name: D_114, dtype: float64
Unique values for D_116: 0    0.0
1    NaN
2    1.0
Name: D_116, dtype: float64
Unique values for D_117: 0    4.0
1   -1.0
2    6.0
3    2.0
4    1.0
5    NaN
6    3.0
7    5.0
Name: D_117, dtype: float64
Unique values for D_120: 0    0.0
1    1.0
2    NaN
Name: D_120, dtype: float64
Unique values for D_126: 0    1.0
1    NaN
2    0.0
3   -1.0
Name: D_126, dtype: float64
Unique values for D_63: 0    CR
1    CO
2    CL
3    XZ
4    XM
5    XL
Name: D_63, dtype: object
Unique values for D_64: 0      O
1      R
2    NaN
3      U
4     -1
Name: D_64, dtype: object
Unique values for D_66: 0    NaN
1    1.0
2    0.0
Name: D_66, dtype: float64
Unique values for D_68: 0    6.0
1    2.0
2    3.0
3    NaN
4    

In [16]:
print('The number of D columns : 95 with 9 categorical columns => going from D_39 to D_145 with the lack of D_40, 57, 67, 85, 90, 95, 97, 98, 99, 100 and 101 (11 columns)')
print('The number of S columns : 22 numerical columns => lack of 5 columns \n (S_2, S_3, S_5, S_6, S_7, S_8, S_9, S_11, S_12, S_13, S_15, S_16, S_17, S_18, S_19, S_20, S_22, S_23, S_24, S_25, S_26, S_27')
print('The number of P columns : 3 numerical columns => P_2, P_3 and P_4 with the lack of P_1. P_3 looks to be a normal distribution')
print('The number of B columns : 40 with 2 categorical columns => going from B_1 to B_42 with the lack of B_34 and B_35')
print('The number of R columns : 3 numerical columns => R_2, R_3 and R_4 with the lack of R_1')

The number of D columns : 95 with 9 categorical columns => going from D_39 to D_145 with the lack of D_40, 57, 67, 85, 90, 95, 97, 98, 99, 100 and 101 (11 columns)
The number of S columns : 22 numerical columns => lack of 5 columns 
 (S_2, S_3, S_5, S_6, S_7, S_8, S_9, S_11, S_12, S_13, S_15, S_16, S_17, S_18, S_19, S_20, S_22, S_23, S_24, S_25, S_26, S_27
The number of P columns : 3 numerical columns => P_2, P_3 and P_4 with the lack of P_1. P_3 looks to be a normal distribution
The number of B columns : 40 with 2 categorical columns => going from B_1 to B_42 with the lack of B_34 and B_35
The number of R columns : 3 numerical columns => R_2, R_3 and R_4 with the lack of R_1


In [12]:
train.head()

,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,...,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-03-09,0.938469,0.001733,0.008724,1.006838,0.009228,0.124035,0.008771,0.004709,...,NaN,NaN,NaN,0.002427,0.003706,0.003818,NaN,0.000569,0.000610,0.002674
1,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-04-07,0.936665,0.005775,0.004923,1.000653,0.006151,0.126750,0.000798,0.002714,...,NaN,NaN,NaN,0.003954,0.003167,0.005032,NaN,0.009576,0.005492,0.009217
2,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-05-28,0.954180,0.091505,0.021655,1.009672,0.006815,0.123977,0.007598,0.009423,...,NaN,NaN,NaN,0.003269,0.007329,0.000427,NaN,0.003429,0.006986,0.002603
3,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-06-13,0.960384,0.002455,0.013683,1.002700,0.001373,0.117169,0.000685,0.005531,...,NaN,NaN,NaN,0.006117,0.004516,0.003200,NaN,0.008419,0.006527,0.009600
4,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-07-16,0.947248,0.002483,0.015193,1.000727,0.007605,0.117325,0.004653,0.009312,...,NaN,NaN,NaN,0.003671,0.004946,0.008889,NaN,0.001670,0.008126,0.009827


In [5]:
train_trainLabel_join = train.merge(train_labels,how='left',on='customer_ID')
catfeats_trainLabel_join = cat_feats.merge(train_labels,how='left',on='customer_ID')

In [13]:
catfeats_trainLabel_join[catfeats_trainLabel_join['target']==1]['B_30'].unique().compute()

0    1.0
1    0.0
2    2.0
3    NaN
Name: B_30, dtype: float64

In [14]:
catfeats_trainLabel_join[catfeats_trainLabel_join['target']==1]['B_38'].unique().compute()

0    5.0
1    6.0
2    7.0
3    1.0
4    4.0
5    3.0
6    2.0
7    NaN
Name: B_38, dtype: float64

In [15]:
catfeats_trainLabel_join[catfeats_trainLabel_join['target']==1]['D_114'].unique().compute()

0    0.0
1    1.0
2    NaN
Name: D_114, dtype: float64

In [16]:
catfeats_trainLabel_join[catfeats_trainLabel_join['target']==1]['D_116'].unique().compute()

0    0.0
1    NaN
2    1.0
Name: D_116, dtype: float64

In [17]:
catfeats_trainLabel_join[catfeats_trainLabel_join['target']==1]['D_117'].unique().compute()

0   -1.0
1    4.0
2    NaN
3    1.0
4    2.0
5    3.0
6    5.0
7    6.0
Name: D_117, dtype: float64

In [18]:
catfeats_trainLabel_join[catfeats_trainLabel_join['target']==1]['D_120'].unique().compute()

0    0.0
1    NaN
2    1.0
Name: D_120, dtype: float64

In [19]:
catfeats_trainLabel_join[catfeats_trainLabel_join['target']==1]['D_126'].unique().compute()

0    1.0
1    0.0
2   -1.0
3    NaN
Name: D_126, dtype: float64

In [41]:
catfeats_trainLabel_join[catfeats_trainLabel_join['target']==1]['D_63'].unique().compute()

0    CO
1    CL
2    CR
3    XM
4    XL
5    XZ
Name: D_63, dtype: object

In [20]:
catfeats_trainLabel_join[catfeats_trainLabel_join['target']==1]['D_64'].unique().compute()

0      U
1      O
2    NaN
3      R
4     -1
Name: D_64, dtype: object

In [21]:
catfeats_trainLabel_join[catfeats_trainLabel_join['target']==1]['D_66'].unique().compute()

0    1.0
1    NaN
2    0.0
Name: D_66, dtype: float64

In [22]:
catfeats_trainLabel_join[catfeats_trainLabel_join['target']==1]['D_68'].unique().compute()

0    3.0
1    6.0
2    5.0
3    NaN
4    4.0
5    1.0
6    2.0
7    0.0
Name: D_68, dtype: float64

In [9]:
catfeats_trainLabel_join[catfeats_trainLabel_join['target']==1].head()

,customer_ID,B_30,B_38,D_114,D_116,D_117,D_120,D_126,D_63,D_64,D_66,D_68,target
104,0000f99513770170a1aba690daeeb8a96da4a39f11fc27...,1.0,5.0,0.0,0.0,-1.0,0.0,1.0,CO,U,1.0,3.0,1
105,0000f99513770170a1aba690daeeb8a96da4a39f11fc27...,1.0,5.0,0.0,0.0,-1.0,0.0,1.0,CO,U,1.0,3.0,1
106,0000f99513770170a1aba690daeeb8a96da4a39f11fc27...,1.0,6.0,0.0,0.0,-1.0,0.0,1.0,CO,U,1.0,6.0,1
107,0000f99513770170a1aba690daeeb8a96da4a39f11fc27...,1.0,7.0,0.0,0.0,-1.0,0.0,1.0,CO,U,1.0,6.0,1
108,0000f99513770170a1aba690daeeb8a96da4a39f11fc27...,1.0,7.0,0.0,0.0,-1.0,0.0,1.0,CO,U,1.0,6.0,1


In [29]:
train_trainLabel_join[train_trainLabel_join['target']==1][['P_2','P_3','P_4','R_2','R_3','R_4']].describe().compute()

,P_2,P_3,P_4,R_2,R_3,R_4
count,1.366093e+06,1.244039e+06,1.377869e+06,1.377869e+06,1.377869e+06,1.377869e+06
mean,3.968602e-01,5.267698e-01,2.834218e-01,1.399179e-01,2.256505e-01,9.253445e-02
std,1.945577e-01,2.253384e-01,4.379168e-01,3.416449e-01,3.161011e-01,2.826330e-01
min,-4.589548e-01,-1.298901e+00,1.009896e-08,2.810394e-09,2.238322e-09,1.618992e-09
25%,3.188087e-01,4.506216e-01,3.881625e-03,3.093204e-03,8.623292e-03,2.923374e-03
50%,4.320343e-01,5.765630e-01,7.614955e-03,6.060022e-03,2.012764e-01,5.677573e-03
75%,5.462155e-01,6.765356e-01,9.587939e-01,9.051380e-03,4.023662e-01,8.464747e-03
max,1.009979e+00,2.139391e+00,1.269390e+00,1.010000e+00,1.160260e+01,1.010000e+00


In [ ]:
train_trainLabel_join[train_trainLabel_join['target']==0][['P_2','P_3','P_4','R_2','R_3','R_4']].describe().compute()